# Feature Engineering
Desarrollo de cálculos y lógica de negocio, tales como:
- KPIs
- Paso de gastos al mercado argentino

## 1. Lectura de archivos limpios e importación de librerias
- Importación de librerías

In [1]:
import sys, os

# Obtengo la ruta raíz del proyecto (subo un nivel desde /notebooks)
project_root = os.path.abspath("../")

# La agrego al path si no está
if project_root not in sys.path:
    sys.path.append(project_root)

# Importado de librerías
from equivalencias.asignar_equivalencias import devolver_clave
from equivalencias.categorias_indec import catego_materias_primas
from equivalencias.equivalencias_pesos import equivalencias_pesos
import pandas as pd
import json

- Lectura de archivos

In [2]:
dir_clean = r'..\data\clean'

# Lectura de CSVs 
df_orders = pd.read_csv(dir_clean + r'\orders.csv')
df_items = pd.read_csv(dir_clean + r'\items.csv')
df_indec = pd.read_csv(dir_clean + r'\indec.csv')

## 2. Paso de gastos al mercado argentino
Como ya se dijo en el `README.md` raíz, generalicé diversas categorías con un diccionario. Esto fue desarrollado con IA para agilizar estas tareas mayormente manuales.\
Por lo tanto, primero me concentro en analizar las sub-categorías contenidas en el dataset `items`, para luego identificar qué materias primas en el informe del `INDEC` se correlacionan mejor con ellas.\
La forma en la que decidí relacionar las categorías con materias primas es asignando una cantidad númerica a cada categoría, como si de vectores se tratara
> Para más información, asistir a `equivalencias\README.md`

- Reviso sub-categorías de platos (las categorías sin indiferentes porque refiere a la carta, no al plato en sí)
- Reviso categorías del `INDEC`

In [3]:
# Subcategorías
sub_categorias_items = df_items['subcategoria'].drop_duplicates().tolist()

print(f'Cantidad de sub-categorías: {len(sub_categorias_items)}\n')
print(sub_categorias_items)

# Materias primas
categorias_indec = df_indec['variedad'].tolist()

print(f'Cantidad de variedades: {len(categorias_indec)}\n')
print(categorias_indec)

Cantidad de sub-categorías: 19

['sopas', 'ensaladas', 'platos_de_aperitivo', 'mariscos', 'platos_de_carne', 'aves_de_corral', 'pastas', 'vegano', 'papas', 'verduras', 'arroz', 'pan', 'tortas', 'helado', 'tartas', 'pudines', 'frutas', 'bebidas_no_alcoholicas', 'bebidas_alcoholicas']
Cantidad de variedades: 55

['pan_frances_tipo_flauta', 'pan_de_mesa', 'galletitas_dulces_envasadas_sin_relleno', 'galletitas_de_agua_envasadas', 'harina_de_trigo_comun_000', 'arroz_blanco_simple', 'fideos_secos_tipo_guisero', 'asado', 'carne_picada_comun', 'paleta', 'cuadril', 'nalga', 'pollo_entero', 'filet_de_merluza_fresco', 'jamon_cocido', 'salchichon', 'salame', 'aceite_de_girasol', 'leche_fresca_entera_en_sachet', 'leche_en_polvo_entera', 'queso_cremoso', 'queso_pategras', 'queso_sardo', 'manteca', 'yogur_firme', 'dulce_de_leche', 'huevos_de_gallina', 'manzana_deliciosa', 'limon', 'naranja', 'banana', 'batata', 'papa', 'cebolla', 'lechuga', 'tomate_redondo', 'zapallo_anco', 'tomate_entero_en_conserva

- Asignación de categorías en el `INDEC` y cálculo promedio de precio

In [4]:
# Asignación de categorías
df_indec['categoria'] = df_indec['variedad'].map(lambda x: devolver_clave(x, catego_materias_primas)[0])

# Cálculo promedio de precio
df_promedios = df_indec.groupby('categoria', as_index=False)['precio'].mean().round(2)

# Paso a diccionario para mayor facilidad de próximas tareas
dict_promedios = dict(zip(df_promedios['categoria'], df_promedios['precio']))

- Verificación rápida entre equivalencias

In [ ]:
for subcategoria, asignacion in equivalencias_pesos.items():
    for categoria, peso in asignacion.items():
        if categoria not in catego_materias_primas:
            print(f'-> {categoria} no se encuentra en el diccionario del INDEC!')

- Enlace entre datasets `INDEC` y `ventas` 

In [ ]:
precios_subcategorias = {}

for row in df_items.itertuples():
    asignacion = equivalencias_pesos.get(row.subcategoria)

    precio_final = sum(
        dict_promedios[categoria] * peso
        for categoria, peso in asignacion.items()
    ) / 5 # -> Siempre suman

    precios_subcategorias[row.subcategoria] = round(precio_final, 2)

{'sopas': 3550.24, 'ensaladas': 1837.9, 'platos_de_aperitivo': 5220.65, 'mariscos': 6011.25, 'platos_de_carne': 10098.79, 'aves_de_corral': 5071.57, 'pastas': 6974.94, 'vegano': 4053.53, 'papas': 2114.31, 'verduras': 1699.69, 'arroz': 8687.28, 'pan': 4907.14, 'tortas': 7694.9, 'helado': 11379.81, 'tartas': 4667.94, 'pudines': 10679.6, 'frutas': 2328.84, 'bebidas_no_alcoholicas': 1481.82, 'bebidas_alcoholicas': 2870.22}


> **Nota**: Se que los DataFrames son pequeños, por lo que ejecutar una triple iteración no resulta de mucho consumo

- Importación de las equivalencias y cálculo de precio promedio

## 5. Cálculo de ganancias 
Tanto en: 
- pesos $ 
- porcentajes %, donde el porcentaje sería en relación con la ganancia total -> ganancia total es el 100%, entonces ganancia neta es X%

In [7]:
# Uno las tablas
df_ventas = df_orders.merge(df_items, on="item", how="inner")

ganancia_total = round(((df_ventas["cantidad"] * df_ventas["precio"]).sum()), 1)
gasto_total = round(((df_ventas["cantidad"] * df_ventas["costo_produccion"]).sum()), 1)

# Cálculo monetario $
ganancia_neta = round((ganancia_total - gasto_total), 1)

# Cálculo percentil %
porcentaje_neto = round(((ganancia_neta * 100) / ganancia_total), 2)

print(f'Las ganancias desde el {df_ventas["fecha_orden"].min()} hasta el {df_ventas["fecha_orden"].max()}:')
print(f' - ${ganancia_neta} pesos')
print(f' - Con un promedio de {porcentaje_neto}% de ganancia por cada venta')

Las ganancias desde el 2022-01-01 hasta el 2022-12-31:
 - $854914832.2 pesos
 - Con un promedio de 15.73% de ganancia por cada venta


## 6. Cálculo de pérdidas
- Utilizando un promedio de pérdidas por fugas de un ~10% (estimado en `data\README.md`) donde está aplicado (el 10%) al costo de los insumos
- Y estimando que con una buena gestión podría reducirse en un 5%

In [8]:
perdida_optimizada = gasto_total * 0.05 
gasto_optimizado = round((gasto_total - (gasto_total * 0.05)), 1)

# Cálculo monetario $
ganancia_neta_opt = round((ganancia_total - gasto_optimizado), 1)

# Cálculo percentil %
porcentaje_neto_opt = round(((ganancia_neta_opt * 100) / ganancia_total), 2)

# Ganancias posibles
diferencia_ganancia = round((ganancia_neta_opt - ganancia_neta), 1)
diferencia_porcentaje = round((porcentaje_neto_opt - porcentaje_neto), 2)

print(f'Las ganancias optmizadas desde el {df_ventas["fecha_orden"].min()} hasta el {df_ventas["fecha_orden"].max()}:')
print(f' - ${ganancia_neta_opt} pesos -> Diferencia de {diferencia_ganancia}')
print(f' - Con un promedio de {porcentaje_neto_opt}% de ganancia por cada venta -> Diferencia del {diferencia_porcentaje}%')

Las ganancias optmizadas desde el 2022-01-01 hasta el 2022-12-31:
 - $1083938088.1 pesos -> Diferencia de 229023255.9
 - Con un promedio de 19.94% de ganancia por cada venta -> Diferencia del 4.21%


In [9]:
kpis = {
    "ganancia_total": ganancia_total,
    "gasto_total": gasto_total,
    "ganancia_neta": ganancia_neta,
    "gasto_optimizado": gasto_optimizado,
    "ganancia_neta_opt": ganancia_neta_opt,
    "porcentaje_neto_opt": porcentaje_neto_opt,
    "diferencia_ganancia": diferencia_ganancia,
    "diferencia_porcentaje": diferencia_porcentaje
}

# Almacenado en JSON
with open(clean_dir + r"\kpis.json", "w") as f:
    json.dump(kpis, f)

NameError: name 'clean_dir' is not defined